<a href="https://colab.research.google.com/github/TiagoIesbick/dashboard-etl/blob/main/dashboard_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import pathlib

In [17]:
file_path = r'/content/drive/MyDrive/Dashboard_data/utils/auxiliary_tables.xlsx'

df_uni = pd.read_excel(file_path, sheet_name='n_uni', skiprows=2, header=None, names=['uni'])
df_proj = pd.read_excel(file_path, sheet_name='n_proj', skiprows=3, header=None, names=['proj', 'nome_proj'])
df_elem = pd.read_excel(file_path, sheet_name='n_elem', skiprows=3, header=None, names=['elem', 'nome_elem'])
df_vinc = pd.read_excel(file_path, sheet_name='n_vinc', skiprows=3, header=None, names=['vinc', 'nome_vinc'])
df_rub = pd.read_excel(file_path, sheet_name='n_rub', skiprows=3, header=None, names=['rub', 'nome_rub'])

df_uni['nome_uni'] = df_uni['uni'].str[5:]
df_uni['uni_int'] = df_uni['uni'].str[:4].astype(int)
df_proj['proj_int'] = df_proj['proj'].str[:4].astype(int)
df_elem['elem_int'] = df_elem['elem'].str[:6].astype(int)
df_rub['rub_int'] = df_rub['rub'].str[:12].astype('int64')
df_vinc['vinc_int'] = df_vinc['vinc'].str[:4].astype(int)

In [18]:
# Generating a dataframe with paid expenses data from 2002 to 2023

sdo_folder_path = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Despesas/SDO_PAGAMENTOS')

df_exp_2002_2023 = pd.DataFrame()

for sdo_file in sdo_folder_path.iterdir():
    df = pd.read_excel(sdo_file)
    df_exp_2002_2023 = pd.concat([df_exp_2002_2023.dropna(axis=1, how='all'), df.dropna(axis=1, how='all')], ignore_index=True)

df_exp_2002_2023.drop(['VLCANCEL','DSFORMAPAG','DTPROG','BANCOCREDOR', 'AGENCIACREDOR','CONTA','NRCHEQUE',
             'IDENTPESSOA', 'NOMEPESSOA', 'BANCOVF', 'AGENCIAVF', 'CONTAVF', 'ORGREQ', 'CODCOMPL',
             'PERCENTUAL', 'VLRESULTADO', 'IRPF_CODCOMPL', 'IRPF_PERCENTUAL', 'IRPF_VLRESULTADO', 'NRPROCCONTR',
             'NRNOTAFISCAL', 'DTENTREMP', 'DTPREVENT', 'LOCALENT', 'DTBASE', 'CODRECFIN', 'CODTIPOPESSOA'], axis=1, inplace=True)

df_exp_2002_2023.rename(columns = {'ANO':'Ano Emp.', 'EMPENHO':'Num. Empenho', 'NRPARCLIQ':'Num.Parc.Liq.',
                             'NRPARCPAG':'Num.Parc.Pag.', 'DSSITPROG':'Situação Pag.', 'DSTPEMP':'Tipo Emp.',
                             'CPTPAG':'Comp.pagto.', 'CPTPAGEST':'Comp.estorno', 'VLPAG':'Val.Pago',
                             'VLPAGEST':'Val.Pago Estorn.', 'VLRETIDO':'Val. Retido', 'RESUL_PAGO':'Result. pago',
                             'UNIDORC':'Unid. Orçam.', 'PROJATIV':'Proj/Ativ', 'RUBRICA':'Rubrica',
                             'DSELEDES':'Descrição da Rubrica', 'VINCORC':'Vinc. Orçam.', 'CPTEMP':'Comp.Empenho',
                             'GORANOPL':'Ano PL', 'GORNRPL':'Num. PL', 'GORNRPARCPL':'Parc. PL',
                             'GORANOOBRAACAO':'Ano Obra/Ação', 'GOROBRAACAO':'Num.Obra/Ação',
                             'NOMERECFIN':'Nome Recurso Financeiro', 'HISTORICO':'Histórico do Empenho',
                             'TPPROCCONTR':'Tipo Processo'}, inplace = True)

df_exp_2002_2023 = df_exp_2002_2023.dropna(subset=['Comp.pagto.'], ignore_index=True)

In [19]:
# Generating a dataframe with revenue collected from 2004 to 2017

revenue_file_2004_2017 = r'/content/drive/MyDrive/Dashboard_data/Receitas/2004_2017/2004_2017_Cubo_Antigo.xlsx'

df_rev_2004_2017 = pd.read_excel(revenue_file_2004_2017)
df_rev_2004_2017.drop(0, inplace=True)
df_rev_2004_2017.rename(columns={'Exercício': 'YEAR',
                               'Mês': 'MONTH',
                               'Rótulos de Linha': 'orgao',
                               'Vínculo Cod': 'vinculo',
                               'N6 Subalinea': 'desdobramento6',
                               'Valor Arrecadado': 'valor_arrecadado',
                               'Valor Orçado': 'valor_orcado'}, inplace=True)
df_rev_2004_2017['Data'] = pd.to_datetime(df_rev_2004_2017[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']] = df_rev_2004_2017[['YEAR', 'orgao', 'MONTH', 'vinculo']].astype('int64')
df_rev_2004_2017['vinculo'] = df_rev_2004_2017['vinculo'].astype('str').replace('400', '0400').replace('6051', '605X')\
  .replace('1', '0001').replace('605X', '6051')

In [36]:
# Generating a dataframe with revenue collected from 2018 to 2023

revenue_folder_2018_2023 = pathlib.Path(r'/content/drive/MyDrive/Dashboard_data/Receitas/2018_2023')

df_rev_2018_2023 = pd.DataFrame()

for revenue_file in revenue_folder_2018_2023.iterdir():
    df = pd.read_excel(revenue_file)
    df_rev_2018_2023 = pd.concat([df_rev_2018_2023.dropna(axis=1, how='all'), df.dropna(axis=1, how='all')], ignore_index=True)

df_rev_2018_2023.drop(['valor_cancelado', 'valor_lancado', 'valor_meta', 'informacao_complementar'], axis=1, inplace=True)
df_rev_2018_2023 = df_rev_2018_2023[df_rev_2018_2023['orgao_raiz']==7000].reset_index(drop=True)
df_rev_2018_2023.rename(columns={'ano': 'YEAR', 'mes': 'MONTH'}, inplace=True)
df_rev_2018_2023['Data'] = pd.to_datetime(df_rev_2018_2023[['YEAR', 'MONTH']].assign(DAY=1))
df_rev_2018_2023['vinculo'] = df_rev_2018_2023['vinculo'].astype('str').replace('400', '0400')
df_rev_2018_2023['rubrica'] = df_rev_2018_2023['rubrica'].astype('str')

df_rev_2018_2023['Natureza da Receita Antiga'] = df_rev_2018_2023['rubrica'].str[:2] + '.' + df_rev_2018_2023['rubrica'].str[2:3]\
  + '.' + df_rev_2018_2023['rubrica'].str[3:4] + '.' + df_rev_2018_2023['rubrica'].str[4:5] + '.' + df_rev_2018_2023['rubrica'].str[5:7]\
  + '.' + df_rev_2018_2023['rubrica'].str[7:8] + '.' + df_rev_2018_2023['rubrica'].str[8:9] + '.' + df_rev_2018_2023['rubrica'].str[9:11]\
  + '.' + df_rev_2018_2023['rubrica'].str[11:13] + '.' + df_rev_2018_2023['rubrica'].str[13:] + '.' + df_rev_2018_2023['vinculo'] + '.0'

In [39]:
df_rev = pd.concat([df_rev_2004_2017, df_rev_2018_2023], ignore_index=True)

In [40]:
df_rev

,orgao,vinculo,desdobramento6,YEAR,MONTH,valor_arrecadado,valor_orcado,Data,orgao_raiz,nome_orgao,...,categoria,origem,especie,desdobramento1,desdobramento2,desdobramento3,tipo,desdobramento4,desdobramento5,Natureza da Receita Antiga
0,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,1,0.00,0.0,2007-01-01,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,2,0.00,0.0,2007-02-01,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,3,42.41,0.0,2007-03-01,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,4,0.00,0.0,2007-04-01,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,5,0.00,0.0,2007-05-01,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27477,7100,6050,12180121020171 - CPSSS do Servidor Civil Inati...,2020,1,627.64,0.0,2020-01-01,7000.0,PREVIMPA E FUNDOS,...,1 - Receitas Correntes,12 - Contribuições,121 - Contribuições Sociais,1218 - Contribuições Sociais Específicas de Es...,121801 - Contr. Serv. Civil p/ Plano de Seg. S...,1218012 - CPSSS do Servidor Civil Inativo,12180121 - CPSSS do Servidor Civil Inativo - P...,1218012102 - CPSSS do Servidor Civil Inativo -...,121801210201 - CPSSS do Servidor Civil Inativo...,12.1.8.0.12.1.0.20.17.1.6050.0
27478,7100,6050,12180121020161 - CPSSS do Servidor Civil Inati...,2020,1,1141.11,0.0,2020-01-01,7000.0,PREVIMPA E FUNDOS,...,1 - Receitas Correntes,12 - Contribuições,121 - Contribuições Sociais,1218 - Contribuições Sociais Específicas de Es...,121801 - Contr. Serv. Civil p/ Plano de Seg. S...,1218012 - CPSSS do Servidor Civil Inativo,12180121 - CPSSS do Servidor Civil Inativo - P...,1218012102 - CPSSS do Servidor Civil Inativo -...,121801210201 - CPSSS do Servidor Civil Inativo...,12.1.8.0.12.1.0.20.16.1.6050.0
27479,7100,0400,12180111030101 - CPSSS do Servidor Civil Ativo...,2020,1,3405.55,0.0,2020-01-01,7000.0,PREVIMPA E FUNDOS,...,1 - Receitas Correntes,12 - Contribuições,121 - Contribuições Sociais,1218 - Contribuições Sociais Específicas de Es...,121801 - Contr. Serv. Civil p/ Plano de Seg. S...,1218011 - CPSSS do Servidor Civil Ativo,12180111 - CPSSS do Servidor Civil Ativo - Pri...,1218011103 - CPSSS do Servidor Civil Ativo - P...,121801110301 - CPSSS do Servidor Civil Ativo -...,12.1.8.0.11.1.0.30.10.1.0400.0
27480,7100,0400,12180111010141 - CPSSS do Servidor Civil Ativo...,2020,1,0.00,0.0,2020-01-01,7000.0,PREVIMPA E FUNDOS,...,1 - Receitas Correntes,12 - Contribuições,121 - Contribuições Sociais,1218 - Contribuições Sociais Específicas de Es...,121801 - Contr. Serv. Civil p/ Plano de Seg. S...,1218011 - CPSSS do Servidor Civil Ativo,12180111 - CPSSS do Servidor Civil Ativo - Pri...,1218011101 - CPSSS do Servidor Civil Ativo - P...,121801110101 - CPSSS do Servidor Civil Ativo -...,12.1.8.0.11.1.0.10.14.1.0400.0


In [38]:
df_rev_2018_2023

,YEAR,MONTH,orgao_raiz,nome_orgao,orgao,nome_orgao.1,rubrica,vinculo,digito,categoria,...,desdobramento2,desdobramento3,tipo,desdobramento4,desdobramento5,desdobramento6,valor_arrecadado,valor_orcado,Data,Natureza da Receita Antiga
0,2018,12,7000,PREVIMPA E FUNDOS,7201,7201 - FUNDO DE REAPARELHAMENTO E APERFEIÇOAM...,13210041010200,6067,0,1 - Receitas Correntes,...,132100,1321004 - Rem. Recursos do Reg. Próprio de Pre...,13210041 - Rem. dos Rec. do Reg. Próprio de Pr...,1321004101 - Remuneração de Recursos Aplicados...,132100410102 - Fundos de Investimentos em Rend...,13210041010200,43039.58,461407.0,2018-12-01,13.2.1.0.04.1.0.10.20.0.6067.0
1,2018,12,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,72100412027100,6050,0,7 - Receitas Correntes Intra-Orçamentárias,...,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210041 - Contr. Patronal de Serv. At. Civil p...,72100412 - Contr. Patronal de Serv. At. Civil ...,7210041202 - Contr.Patr.Serv.At.Civil p/ RPPS ...,721004120271,72100412027100,0.00,0.0,2018-12-01,72.1.0.0.41.2.0.27.10.0.6050.0
2,2018,12,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,72100412027100,6069,0,7 - Receitas Correntes Intra-Orçamentárias,...,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210041 - Contr. Patronal de Serv. At. Civil p...,72100412 - Contr. Patronal de Serv. At. Civil ...,7210041202 - Contr.Patr.Serv.At.Civil p/ RPPS ...,721004120271,72100412027100,0.00,0.0,2018-12-01,72.1.0.0.41.2.0.27.10.0.6069.0
3,2018,12,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,72100422011100,0400,0,7 - Receitas Correntes Intra-Orçamentárias,...,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210042,72100422 - Contribuição do Servidor Ativo Civi...,7210042201 - Contr. do Serv. At. Civil p/ RPPS...,721004220111,72100422011100,0.00,0.0,2018-12-01,72.1.0.0.42.2.0.11.10.0.0400.0
4,2018,12,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,72100422023100,6050,0,7 - Receitas Correntes Intra-Orçamentárias,...,721004 - Contr. p/ Regime Próprio de Prev. Soc...,7210042,72100422 - Contribuição do Servidor Ativo Civi...,7210042202 - Contrib. dos Servidores - At. Civ...,721004220231,72100422023100,0.00,0.0,2018-12-01,72.1.0.0.42.2.0.23.10.0.6050.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18069,2020,1,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,12180121020171,6050,0,1 - Receitas Correntes,...,121801 - Contr. Serv. Civil p/ Plano de Seg. S...,1218012 - CPSSS do Servidor Civil Inativo,12180121 - CPSSS do Servidor Civil Inativo - P...,1218012102 - CPSSS do Servidor Civil Inativo -...,121801210201 - CPSSS do Servidor Civil Inativo...,12180121020171 - CPSSS do Servidor Civil Inati...,627.64,0.0,2020-01-01,12.1.8.0.12.1.0.20.17.1.6050.0
18070,2020,1,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,12180121020161,6050,0,1 - Receitas Correntes,...,121801 - Contr. Serv. Civil p/ Plano de Seg. S...,1218012 - CPSSS do Servidor Civil Inativo,12180121 - CPSSS do Servidor Civil Inativo - P...,1218012102 - CPSSS do Servidor Civil Inativo -...,121801210201 - CPSSS do Servidor Civil Inativo...,12180121020161 - CPSSS do Servidor Civil Inati...,1141.11,0.0,2020-01-01,12.1.8.0.12.1.0.20.16.1.6050.0
18071,2020,1,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,12180111030101,0400,0,1 - Receitas Correntes,...,121801 - Contr. Serv. Civil p/ Plano de Seg. S...,1218011 - CPSSS do Servidor Civil Ativo,12180111 - CPSSS do Servidor Civil Ativo - Pri...,1218011103 - CPSSS do Servidor Civil Ativo - P...,121801110301 - CPSSS do Servidor Civil Ativo -...,12180111030101 - CPSSS do Servidor Civil Ativo...,3405.55,0.0,2020-01-01,12.1.8.0.11.1.0.30.10.1.0400.0
18072,2020,1,7000,PREVIMPA E FUNDOS,7100,7100 - DEP. MUNICIPAL DE PREV. DOS SERV. PÚBL...,12180111010141,0400,0,1 - Receitas Correntes,...,121801 - Contr.

In [33]:
df_rev_2018_2023['vinculo'].unique()

array(['6067', '6050', '6069', '0400', '6060', '6049'], dtype=object)

In [23]:
df_rev_2004_2017

,orgao,vinculo,desdobramento6,YEAR,MONTH,valor_arrecadado,valor_orcado,Data
1,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,1,0.00,0.00,2007-01-01
2,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,2,0.00,0.00,2007-02-01
3,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,3,42.41,0.00,2007-03-01
4,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,4,0.00,0.00,2007-04-01
5,7000,0001,19122902 - Multas e Juros de Mora das Contribu...,2007,5,0.00,0.00,2007-05-01
...,...,...,...,...,...,...,...,...
9404,7201,6067,13281000 - Remuneração dos Investimentos Em Re...,2017,8,61393.81,17841.66,2017-08-01
9405,7201,6067,13281000 - Remuneração dos Investimentos Em Re...,2017,9,57236.33,17841.67,2017-09-01
9406,7201,6067,13281000 - Remuneração dos Investimentos Em Re...,2017,10,62636.51,17841.66,2017-10-01
9407,7201,6067,13281000 - Remuneração dos Investimentos Em Re...,2017,11,58037.88,17841.68,2017-11-01


In [11]:
df_rev_2004_2017['vinculo'].unique()

array(['0001', '0400', '6050', '6051', '6060', '6069', '6067'],
      dtype=object)

,Rótulos de Linha,Vínculo Cod,N6 Subalinea,Exercício,Mês,Valor Arrecadado,Valor Orçado
0,,NaN,NaN,NaN,NaN,NaN,NaN
1,7000,1.0,19122902 - Multas e Juros de Mora das Contribu...,2007.0,1.0,0.00,0.00
2,7000,1.0,19122902 - Multas e Juros de Mora das Contribu...,2007.0,2.0,0.00,0.00
3,7000,1.0,19122902 - Multas e Juros de Mora das Contribu...,2007.0,3.0,42.41,0.00
4,7000,1.0,19122902 - Multas e Juros de Mora das Contribu...,2007.0,4.0,0.00,0.00
...,...,...,...,...,...,...,...
9404,7201,6067.0,13281000 - Remuneração dos Investimentos Em Re...,2017.0,8.0,61393.81,17841.66
9405,7201,6067.0,13281000 - Remuneração dos Investimentos Em Re...,2017.0,9.0,57236.33,17841.67
9406,7201,6067.0,13281000 - Remuneração dos Investimentos Em Re...,2017.0,10.0,62636.51,17841.66
9407,7201,6067.0,13281000 - Remuneração dos Investimentos Em Re...,2017.0,11.0,58037.88,17841.68
